# Stage 1 - Data Cleaning and Manipulation with Pandas

In this stage, we will start with a dataset call with our name. This dataset has information about Forbes Billionaires 2018. 

## 1. Import the dataset 

In [1]:
from sqlalchemy import create_engine
import sqlite3
import pandas as pd
import re

conexion = sqlite3.connect('/Users/Natalio/Desktop/Cibeles/Ironhack/Ironhack-Module-1-Project---Cibeles-Redondo/data/CibelesR.db')

In [2]:
# I am going to create different dataframes for every table in my database.

df_business_info = pd.read_sql_query('SELECT * FROM business_info', conexion)
df_personal_info = pd.read_sql_query('SELECT * FROM personal_info', conexion)
df_rank_info = pd.read_sql_query('SELECT * FROM rank_info', conexion)

df_intermediate = pd.merge(df_business_info, df_personal_info, on = 'id') # I try to make a merge with all the df but I was not able to.
df_project = pd.merge(df_intermediate, df_rank_info, on = 'id')

#display(df_business_info.head(11))
#display(df_personal_info.head(11))
#display(df_rank_info.head(11))
display(df_project.head(11))

,id,Unnamed: 0_x,Source,worth,worthChange,realTimeWorth,realTimePosition,lastName,age,Unnamed: 0_y,gender,country,image,name,position,Unnamed: 0
0,1307,52,Technology ==> Amazon,112.0 BUSD,0.0 millions USD,None,1,BEZOS,54 years old,52,Male,None,https://specials-images.forbesimg.com/imageser...,jefF BEZOs,1.0,52
1,6752,53,Technology ==> Microsoft,90.0 BUSD,-0.001 millions USD,None,2,gATES,62 years old,53,Male,None,https://specials-images.forbesimg.com/imageser...,BILL GATes,2.0,53
2,3651,54,Finance and Investments ==> Berkshire Hathaway,84.0 BUSD,-0.002 millions USD,None,3,bufFETT,87 years old,54,M,None,https://specials-images.forbesimg.com/imageser...,WARREN BUFFett,3.0,54
3,8849,55,Fashion & Retail ==> LVMH,72.0 BUSD,0.0 millions USD,None,4,ARNAUlt,69 years old,55,M,France,https://specials-images.forbesimg.com/imageser...,bernarD ARNAULT,4.0,55
4,4656,56,Technology ==> Facebook,71.0 BUSD,0.0 millions USD,None,5,zUCKERBERG,1985,56,M,United States,https://specials-images.forbesimg.com/imageser...,mark zUCKerberg,5.0,56
5,9545,57,Fashion & Retail ==> Zara,70.0 BUSD,0.0 millions USD,None,6,orTEGa,82 years old,57,M,Spain,https://specials-images.forbesimg.com/imageser...,aMANCIO ORTEGA,6.0,57
6,2460,58,Telecom ==> telecom,67.1 BUSD,0.001 millions USD,None,7,slim Helu,78 years old,58,M,None,https://specials-images.forbesimg.com/imageser...,carLOS SLIM helu,7.0,58
7,3627,59,Diversified ==> Koch Industries,60.0 BUSD,0.0 millions USD,None,12,koCH,82 years old,59,M,United States,https://specials-images.forbesimg.com/imageser...,chARLES KOCH,8.0,59
8,6421,60,Diversified ==> Koch Industries,60.0 BUSD,0.0 millions USD,None,13,kOCH,78 years old,60,M,None,https://specials-images.forbesimg.com/imageser...,daVID Koch,9.0,60
9,3817,61,Technology ==> software,58.5 BUSD,-0.001 millions USD,None,8,elLISON,73 years old,61,M,USA,https://specials-images.forbesimg.com/imageser...,larrY ELLISon,10.0,61


In [3]:
# We are reading correctly the information in the DDBB, so we can start to depurate de original data.

## 2. Data Cleaning

### 2.1. Data Cleaning: business_info table

In [4]:
# Firts of all we are going to take an overal look about the data in this table
# We are going to know what time of data we have in every single feature

df_project.info(memory_usage = 'deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2208 entries, 0 to 2207
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                2208 non-null   int64  
 1   Unnamed: 0_x      2208 non-null   int64  
 2   Source            2208 non-null   object 
 3   worth             2208 non-null   object 
 4   worthChange       2208 non-null   object 
 5   realTimeWorth     0 non-null      object 
 6   realTimePosition  2208 non-null   int64  
 7   lastName          2208 non-null   object 
 8   age               2143 non-null   object 
 9   Unnamed: 0_y      2208 non-null   int64  
 10  gender            2195 non-null   object 
 11  country           2208 non-null   object 
 12  image             2208 non-null   object 
 13  name              2208 non-null   object 
 14  position          2208 non-null   float64
 15  Unnamed: 0        2208 non-null   int64  
dtypes: float64(1), int64(5), object(10)
memory

In [5]:
# It is look like we have the same number of non-null data in all the columns. Also, it is remarcable that some columns
# with names like 'worth' are 'object' type and not numbers.

In [6]:
df_business_info.describe(include = 'all')

,id,Unnamed: 0,Source,worth,worthChange,realTimeWorth,realTimePosition
count,2208.000000,2208.000000,2208,2208,2208,0,2208.000000
unique,NaN,NaN,892,192,95,0,NaN
top,NaN,NaN,Real Estate ==> real estate,1.2 BUSD,0.0 millions USD,NaN,NaN
freq,NaN,NaN,172,132,1790,NaN,NaN
mean,5511.740036,1155.500000,NaN,NaN,NaN,NaN,1102.043478
std,2631.468022,637.539018,NaN,NaN,NaN,NaN,630.218047
min,1000.000000,52.000000,NaN,NaN,NaN,NaN,1.000000
25%,3187.750000,603.750000,NaN,NaN,NaN,NaN,555.750000
50%,5513.500000,1155.500000,NaN,NaN,NaN,NaN,1106.500000
75%,7801.500000,1707.250000,NaN,NaN,NaN,NaN,1655.250000


In [7]:
# Deleting duplicate data
original_leng_business_info = len(df_project)
df_project_v2 = df_project.drop_duplicates()
no_duplicates_leng_df_project = len(df_project_v2)

print(f'Duplicate records: {original_leng_business_info - no_duplicates_leng_df_project}')

Duplicate records: 0


In [8]:
# Now, we are going to detect the null data in our dataframe

In [9]:
null_cols_df_project = df_project.isnull().sum()
null_cols_df_project

id                     0
Unnamed: 0_x           0
Source                 0
worth                  0
worthChange            0
realTimeWorth       2208
realTimePosition       0
lastName               0
age                   65
Unnamed: 0_y           0
gender                13
country                0
image                  0
name                   0
position               0
Unnamed: 0             0
dtype: int64

In [10]:
# We only find null data in column 'realTimeWorth'. Let's check it out

In [11]:
df_project['realTimeWorth'].head(20)

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
Name: realTimeWorth, dtype: object

In [12]:
# We are not going to use realTimeWorth. Also, we are not going to use Unnamed from the different datasets
df_project.columns
df_project = df_project[['id', 'Source', 'worth','realTimePosition', 'lastName', 'age', 'gender',
       'country', 'image', 'name', 'position']]

# All with the same style of naming
df_project.columns = ['id', 'source', 'worth',
       'realtimeposition', 'lastname', 'age', 'gender',
       'country', 'image', 'name', 'position']
df_project

,id,source,worth,realtimeposition,lastname,age,gender,country,image,name,position
0,1307,Technology ==> Amazon,112.0 BUSD,1,BEZOS,54 years old,Male,None,https://specials-images.forbesimg.com/imageser...,jefF BEZOs,1.0
1,6752,Technology ==> Microsoft,90.0 BUSD,2,gATES,62 years old,Male,None,https://specials-images.forbesimg.com/imageser...,BILL GATes,2.0
2,3651,Finance and Investments ==> Berkshire Hathaway,84.0 BUSD,3,bufFETT,87 years old,M,None,https://specials-images.forbesimg.com/imageser...,WARREN BUFFett,3.0
3,8849,Fashion & Retail ==> LVMH,72.0 BUSD,4,ARNAUlt,69 years old,M,France,https://specials-images.forbesimg.com/imageser...,bernarD ARNAULT,4.0
4,4656,Technology ==> Facebook,71.0 BUSD,5,zUCKERBERG,1985,M,United States,https://specials-images.forbesimg.com/imageser...,mark zUCKerberg,5.0
...,...,...,...,...,...,...,...,...,...,...,...
2203,5874,"Fashion & Retail ==> fashion, entertainment",1.0 BUSD,2134,ZHAO,51 years old,None,China,https://specials-images.forbesimg.com/imageser...,zhaO XIAOQiang,2204.0
2204,8914,Manufacturing ==> electrical equipment,1.0 BUSD,2134,ZHOU,55 years old,M,China,https://specials-images.forbesimg.com/imageser...,ZHOU LIANgzhang,2205.0
2205,7431,Manufacturing ==> electrical equipment,1.0 BUSD,1856,ZHU,51 years old,M,China,https://specials-images.forbesimg.com/imageser...,zhu xiNGMing,2206.0
2206,2966,Manufacturing ==> printed circuit boards,1.0 BUSD,1978,ZHUO,52 years old,F,Hong Kong,https://specials-images.forbesimg.com/imageser...,ZHUO JUn,2207.0


In [13]:
# We are going to focus on the womens in the df. First, we are going to clean the gender column
df_project['gender'].unique()

array(['Male', 'M', 'None', None, 'F', 'Female'], dtype=object)

In [14]:
df_project['gender'].replace('F', 'Female', inplace = True)

#We can find the 'None' gender later with scraping

df_project_fem = df_project[df_project['gender'] == 'Female']
df_project_fem['lastname'] = df_project_fem['lastname'].str.capitalize()
# not working df_project_fem['name'] = [i.str.capitalize() for i in df_project_fem['name']]
df_project_fem['name'] = df_project_fem['name'].str.capitalize()
df_project_fem


/Users/Natalio/miniconda3/envs/Ironhack_env/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/Natalio/miniconda3/envs/Ironhack_env/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,id,source,worth,realtimeposition,lastname,age,gender,country,image,name,position
33,2434,"Food and Beverage ==> candy, pet food",23.6 BUSD,34,Mars,78 years old,Female,None,https://specials-images.forbesimg.com/imageser...,Jacqueline mars,34.0
42,5201,Real Estate ==> real estate,21.9 BUSD,31,Yang,1983,Female,People's Republic of China,https://specials-images.forbesimg.com/imageser...,Yang huiyan,43.0
57,4720,"Philanthropy/NGO ==> Apple, Disney",18.8 BUSD,48,Powell jobs,54 years old,Female,None,https://specials-images.forbesimg.com/imageser...,Laurene powell jobs,58.0
70,5294,Metals & Mining ==> mining,17.4 BUSD,66,Rinehart,64 years old,Female,None,https://specials-images.forbesimg.com/imageser...,Gina rinehart,71.0
80,7565,Metals & Mining ==> mining,16.3 BUSD,70,Fontbona,75 years old,Female,None,https://specials-images.forbesimg.com/imageser...,Iris fontbona,81.0
...,...,...,...,...,...,...,...,...,...,...,...
2106,3694,"Diversified ==> insurance, NFL team",1.1 BUSD,1966,Tisch,67 years old,Female,USA,https://specials-images.forbesimg.com/imageser...,Laurie tisch,2107.0
2125,1652,Fashion & Retail ==> Spanx,1.0 BUSD,2127,Blakely,47 years old,Female,United States,https://specials-images.forbesimg.com/imageser...,Sara blakely,2126.0
2147,3882,Technology ==> printed circuit boards,1.0 BUSD,1980,Huang,56 years old,Female,None,https://specials-images.forbesimg.com/imageser...,Huang xiaofen,2148.0
2182,3344,Manufacturing ==> biotech,1.0 BUSD,1815,Schwartz,91 years old,Female,USA,https://specials-images.forbesimg.com/imageser...,Alice schwartz,2183.0


In [15]:
prueba = pd.DataFrame(df_project_fem.source.str.split('==>',1).tolist(), columns = ['industry', 'company'])

In [16]:
# Cleaning the source to understand if there is a industry were the womens are best of business
def splitting(df):
    return df.split('  ==> ')

df_project_fem['source'] = df_project_fem['source'].apply(splitting)
df_project_fem['industry'] = re.findall('')

/Users/Natalio/miniconda3/envs/Ironhack_env/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


TypeError: findall() missing 1 required positional argument: 'string'

In [17]:
df_project_fem['source'][0]
#df_project_fem['industry'] = 

KeyError: 0

In [18]:
df_project_fem['country'].unique()

array(['None', "People's Republic of China", 'USA', 'Italy', 'China',
       'United States', 'Mexico', 'Switzerland', 'Denmark', 'Hong Kong',
       'Germany', 'Portugal', 'India', 'Kazakhstan', 'Norway',
       'United Kingdom', 'Sweden', 'Brazil', 'Australia', 'South Korea',
       'Cyprus', 'Japan'], dtype=object)

In [19]:
df_project_fem = df_project_fem[df_project_fem['country'] != str('None')]
df_project_fem.rename(columns = {'worth':'worth BUSD'}, inplace = True)
df_project_fem['worth BUSD'] = df_project_fem['worth BUSD'].str.replace(' BUSD', '').astype('float')


/Users/Natalio/miniconda3/envs/Ironhack_env/lib/python3.7/site-packages/pandas/core/frame.py:4130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/Users/Natalio/miniconda3/envs/Ironhack_env/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [20]:
df_project_fem['born year'] = df_project_fem['age'].values
#2018 is the year of the dataset, so I will deduce the age or the year of born with the data
df_project_fem['age'] = df_project_fem['age'].str.replace('years old', '').astype('float')



/Users/Natalio/miniconda3/envs/Ironhack_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/Natalio/miniconda3/envs/Ironhack_env/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [21]:
#df_project_fem['age'] = [2018 - i if (i.astype(str).map(len)) >= 4 else i for i in df_project_fem['age']]

df_project_fem['age'] = [2018 - i if i >= 1000 else i for i in df_project_fem['age']]

df_project_fem

/Users/Natalio/miniconda3/envs/Ironhack_env/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,id,source,worth BUSD,realtimeposition,lastname,age,gender,country,image,name,position,born year
42,5201,"[Real Estate, real estate]",21.9,31,Yang,35.0,Female,People's Republic of China,https://specials-images.forbesimg.com/imageser...,Yang huiyan,43.0,1983
179,7158,"[Media & Entertainment, media]",8.7,270,Taylor,76.0,Female,USA,https://specials-images.forbesimg.com/imageser...,Margaretta taylor,180.0,76 years old
196,5161,"[Healthcare, pharmaceuticals]",7.9,215,Landini aleotti,75.0,Female,Italy,https://specials-images.forbesimg.com/imageser...,Massimiliana landini aleotti,197.0,75 years old
203,2375,"[Real Estate, real estate]",7.7,186,Wu,54.0,Female,China,https://specials-images.forbesimg.com/imageser...,Wu yajun,204.0,54 years old
215,6435,"[Fashion & Retail, Walmart]",7.3,259,Walton,69.0,Female,United States,https:http://specials-images.forbesimg.com/ima...,Christy walton,216.0,69 years old
...,...,...,...,...,...,...,...,...,...,...,...,...
2095,3188,"[Service, staffing]",1.1,2111,Shinohara,83.0,Female,Japan,https://specials-images.forbesimg.com/imageser...,Yoshiko shinohara,2096.0,83 years old
2106,3694,"[Diversified, insurance, NFL team]",1.1,1966,Tisch,67.0,Female,USA,https://specials-images.forbesimg.com/imageser...,Laurie tisch,2107.0,67 years old
2125,1652,"[Fashion & Retail, Spanx]",1.0,2127,Blakely,47.0,Female,United States,https://specials-images.forbesimg.com/imageser...,Sara blakely,2126.0,47 years old
2182,3344,"[Manufacturing, biotech]",1.0,1815,Schwartz,91.0,Female,USA,https://specials-images.forbesimg.com/imageser...,Alice schwartz,2183.0,91 years old


In [22]:
df_project_fem['born year'] = df_project_fem['born year'].str.replace('years old', '').astype('float')
df_project_fem['born year'] = [2018 - i if i < 1000 else i for i in df_project_fem['born year']]

df_project_fem

#Try to change float to int but it is not working

/Users/Natalio/miniconda3/envs/Ironhack_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/Natalio/miniconda3/envs/Ironhack_env/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,id,source,worth BUSD,realtimeposition,lastname,age,gender,country,image,name,position,born year
42,5201,"[Real Estate, real estate]",21.9,31,Yang,35.0,Female,People's Republic of China,https://specials-images.forbesimg.com/imageser...,Yang huiyan,43.0,1983.0
179,7158,"[Media & Entertainment, media]",8.7,270,Taylor,76.0,Female,USA,https://specials-images.forbesimg.com/imageser...,Margaretta taylor,180.0,1942.0
196,5161,"[Healthcare, pharmaceuticals]",7.9,215,Landini aleotti,75.0,Female,Italy,https://specials-images.forbesimg.com/imageser...,Massimiliana landini aleotti,197.0,1943.0
203,2375,"[Real Estate, real estate]",7.7,186,Wu,54.0,Female,China,https://specials-images.forbesimg.com/imageser...,Wu yajun,204.0,1964.0
215,6435,"[Fashion & Retail, Walmart]",7.3,259,Walton,69.0,Female,United States,https:http://specials-images.forbesimg.com/ima...,Christy walton,216.0,1949.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2095,3188,"[Service, staffing]",1.1,2111,Shinohara,83.0,Female,Japan,https://specials-images.forbesimg.com/imageser...,Yoshiko shinohara,2096.0,1935.0
2106,3694,"[Diversified, insurance, NFL team]",1.1,1966,Tisch,67.0,Female,USA,https://specials-images.forbesimg.com/imageser...,Laurie tisch,2107.0,1951.0
2125,1652,"[Fashion & Retail, Spanx]",1.0,2127,Blakely,47.0,Female,United States,https://specials-images.forbesimg.com/imageser...,Sara blakely,2126.0,1971.0
2182,3344,"[Manufacturing, biotech]",1.0,1815,Schwartz,91.0,Female,USA,https://specials-images.forbesimg.com/imageser...,Alice schwartz,2183.0,1927.0


## Prepare it to be analyzed, and then export it as a clean CSV data file
#### We are goint to analyce the impact of women in Forbes List, how representative they are and where they are from.

In [23]:
df_project_fem.to_csv(f'/Users/Natalio/Desktop/Cibeles/Ironhack/Ironhack-Module-1-Project---Cibeles-Redondo/data/CibelesR_clean_output.csv', index = False)